<center>

# Part 1 Data Collection Capstone


</center>



> **Predicting Footballers Market Values**
   
> **Using 2008 - 2017 data from transfermarkt.co.uk**
   
> **Christopher Williams, ATX, GA DSI Feb 2019**



> **The data science process.**

>1. Define the problem.
>2. Gather the data.
>3. Explore the data.
>4. Model the data.
>5. Evaluate the model.
>6. Answer the problem.


On this jupyter notebook, I'm going to address part 2 of the data science process: gathering the data. See Capstone Model jupyter notebook for the rest of my capstone's data science process.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-1-Data-Collection-Capstone" data-toc-modified-id="Part-1-Data-Collection-Capstone-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 1 Data Collection Capstone</a></span></li><li><span><a href="#Scrapper-Function-get_data" data-toc-modified-id="Scrapper-Function-get_data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scrapper Function get_data</a></span></li><li><span><a href="#Leagues-to-Scrap" data-toc-modified-id="Leagues-to-Scrap-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Leagues to Scrap</a></span></li><li><span><a href="#Function-to-scrape-each-team's-pages-on-each-league-site-to-get-url-&amp;-id" data-toc-modified-id="Function-to-scrape-each-team's-pages-on-each-league-site-to-get-url-&amp;-id-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Function to scrape each team's pages on each league site to get url &amp; id</a></span></li><li><span><a href="#Get-the-Club-Name,-Unique-ID,-and-Name-in-URL-for-Each-League's-Team" data-toc-modified-id="Get-the-Club-Name,-Unique-ID,-and-Name-in-URL-for-Each-League's-Team-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Get the Club Name, Unique ID, and Name in URL for Each League's Team</a></span></li><li><span><a href="#Build-out-the-function-to-insert-team-name-and-team-id-into-url-to-scrap-stats-and-market-value-page" data-toc-modified-id="Build-out-the-function-to-insert-team-name-and-team-id-into-url-to-scrap-stats-and-market-value-page-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Build out the function to insert team name and team id into url to scrap stats and market value page</a></span></li><li><span><a href="#List-of-Years" data-toc-modified-id="List-of-Years-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>List of Years</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Currently-Using-This-One" data-toc-modified-id="Currently-Using-This-One-7.0.0.1"><span class="toc-item-num">7.0.0.1&nbsp;&nbsp;</span>Currently Using This One</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Base-Urls" data-toc-modified-id="Base-Urls-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Base Urls</a></span></li><li><span><a href="#CHANGE-HERE-TO-GET-DATA-FROM-MORE-YEARS" data-toc-modified-id="CHANGE-HERE-TO-GET-DATA-FROM-MORE-YEARS-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>CHANGE HERE TO GET DATA FROM MORE YEARS</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Break" data-toc-modified-id="Break-9.0.0.1"><span class="toc-item-num">9.0.0.1&nbsp;&nbsp;</span>Break</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Data-Retrieval-Function-called-get_data-that-takes-two-urls" data-toc-modified-id="Data-Retrieval-Function-called-get_data-that-takes-two-urls-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Data Retrieval Function called get_data that takes two urls</a></span></li><li><span><a href="#For-2013-2017-data-scrap" data-toc-modified-id="For-2013-2017-data-scrap-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>For 2013-2017 data scrap</a></span></li><li><span><a href="#For-2008-2012-data-scrap" data-toc-modified-id="For-2008-2012-data-scrap-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>For 2008-2012 data scrap</a></span></li><li><span><a href="#END" data-toc-modified-id="END-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>END</a></span></li></ul></div>

In [45]:
# Imports

import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
from time import sleep
from selenium import webdriver
import os
import sys
import sqlalchemy
import re
import psycopg2 as pg2


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)

# Scrapper Function get_data  
- Takes two soup objects from both the Market Cap and Player Stat pages

In [69]:
# get_data scrapper function
# This is the function that gets the data

def get_data(soup1, soup2):

    ################# from the market value page #################

    rows1 = []

    df_url1 = pd.DataFrame(columns=['player', 'current_team', 'current_year','birthdate_age', 'nationality', 'height_m', 'foot', 'date_joined', 'year_joined','signed_from', 'transfer_fee', 'contract_until', 'market_value'])

    current_team = soup1.find_all('a', {'href':True})[10]['href'].split('/')[3]

    current_year = soup1.find_all('a', {'href':True})[10]['href'].split('/')[8]




    for i in soup1.find_all('tr', {'class':['even', 'odd']}):


        try:
            player = i.find('td', {'class':'posrela'}).find_all('tr')[0].find('span',{"class":'hide-for-small'}).text
        except:
            player = '-'

        try:
            birthdate_age = i.find_all('td', {'class':'zentriert'})[1].text
        except:
            birthdate_age = '-'

        try:
            nationality = i.find_all('td')[6].find('img').attrs['title'] 
        except:
            nationality = '-'

        try:
            height_m = i.find_all('td')[8].text
        except:
            height_m = '-'

        try:
            foot = i.find_all('td')[9].text
        except:
            foot = '-'

        try:
            date_joined = i.find_all('td')[10].text
        except:
            date_joined = '-'

        try:
            year_joined = i.find_all('td')[11].find('a')['href'].split('/')[6]
        except:
            year_joined = '-'

        try:
            signed_from = i.find_all('img')[3]['alt'] 
        except:
            signed_from = '-'

        try:
            transfer_fee = i.find_all('img')[3]['title'].split(': ')[1].split(' ')[1]
        except:
            transfer_fee = '-'

        try:
            contract_until = i.find_all('td')[12].text
        except:
            contract_until = '-'

        try:
            #market_value = i.find_all('td')[13].text.split('m')[0]
            market_value = i.find_all('td')[13].text.split('/')[0]
            
        except:
            market_value = '-'

        df_url1.loc[len(df_url1)] = [player, current_team, current_year, birthdate_age, nationality, height_m, foot, date_joined, year_joined, signed_from, transfer_fee, contract_until, market_value]


        rows1.append(i)
    df_url1.head(25)


    ################# from the players stats page #################


    rows2 = []

    df_url2 = pd.DataFrame(columns=['number', 'player', 'position', 'general_position', 'age', 'in_squad', 'appearances', 'goals', 'assists', 'yellow_card', 'second_yellow', 'red_cards', 'substituted_on', 'substituted_off', 'ppm', 'minutes_played'])

    for i in soup2.find_all('tr', {'class':['even', 'odd']}):

        try:
            number = i.find('div', {'class': 'rn_nummer'}).text
        except:
            number = '-'

        try:
            player = i.find('td', {'class':'posrela'}).find_all('tr')[0].find('span',{"class":'hide-for-small'}).text
        except:
            player = '-'

        try:
            position = i.find('td', {'class':'posrela'}).find_all('tr')[1].text
        except:
            position = '-'

        try:
            general_position = i.find_all('td')[0]['title']
        except:
            general_position = '-'

        try:
            age = i.find_all('td', {'class':'zentriert'})[1].text
        except:
            age = '-'


        try:
            in_squad = i.find_all('td')[7].text
        except:
            in_squad = '-'

        try:
            appearances = i.find_all('td')[8].text
        except:
            appearances = '-'

        try:
            goals = i.find_all('td')[9].text
        except:
            goals = '-'

        try:
            assists = i.find_all('td')[10].text
        except:
            assists = '-'

        try:
            yellow_card = i.find_all('td')[11].text
        except:
            yellow_card = '-'

        try:
            second_yellow = i.find_all('td')[12].text
        except:
            second_yellow = '-'

        try:
            red_cards = i.find_all('td')[13].text
        except:
            red_cards = '-'

        try:
            substituted_on = i.find_all('td')[14].text
        except:
            substituted_on = '-'

        try:
            substituted_off = i.find_all('td')[15].text
        except:
            substituted_off = '-'

        try:
            ppm = i.find_all('td')[16].text
        except:
            ppm = '-'

        try:
            minutes_played = i.find_all('td')[17].text
        except:
            minutes_played = '-'


        df_url2.loc[len(df_url2)] = [number, player, position, general_position, age, in_squad, appearances, goals, assists, yellow_card, second_yellow, red_cards, substituted_on, substituted_off, ppm, minutes_played]


        rows2.append(i)
    df_url2.head(25)


    df = pd.merge(df_url1, df_url2, how = 'inner', on = 'player')
    return df

# Leagues to Scrap

In [2]:
# List of all Teams in La Liga
la_liga_url = "https://www.transfermarkt.co.uk/primera-division/startseite/wettbewerb/ES1"

# List of all Teams in Bundelsliga
bundesliga_url = "https://www.transfermarkt.co.uk/1-bundesliga/startseite/wettbewerb/L1"

# List of all Teams in Serie A
serie_a_url = "https://www.transfermarkt.co.uk/serie-a/startseite/wettbewerb/IT1"

# List of all Teams in French Ligue 1
ligue_1_url = "https://www.transfermarkt.co.uk/ligue-1/startseite/wettbewerb/FR1"

# List of all Teams in Premier League
premier_league_url = "https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1"



#### List of all League's teams urls    #####
league_urls = [la_liga_url, bundesliga_url, serie_a_url, ligue_1_url, premier_league_url]

In [3]:
for i in league_urls: 
    print(i)

https://www.transfermarkt.co.uk/primera-division/startseite/wettbewerb/ES1
https://www.transfermarkt.co.uk/1-bundesliga/startseite/wettbewerb/L1
https://www.transfermarkt.co.uk/serie-a/startseite/wettbewerb/IT1
https://www.transfermarkt.co.uk/ligue-1/startseite/wettbewerb/FR1
https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1


#  Function to scrape each team's pages on each league site to get url & id

In [4]:
# Define function to scrape each team's pages on each league site to get url & id

def leagues_teams_scraper(soup_object):
     
    for i in soup.find_all('a', {'class': "vereinprofil_tooltip tooltipstered"}):
      
        ### get the club name ###
        club_name = i.text
      
        ### get the club name used in the url ###
        url_name = i['href'].split('/')[1]
      
        ### get the club ids used in the url ###
        id_number = i['href'].split('/')[4]

        sleep(.01)
        df_leagues_teams_names_urls_and_ids.loc[len(df_leagues_teams_names_urls_and_ids)] = [club_name, id_number, url_name]


# Get the Club Name, Unique ID, and Name in URL for Each League's Team


In [47]:
# Create dataframe called: df_leagues_teams_names_urls_and_ids
# Scraps team's pages on each league site to get club info: to df_leagues_teams_names_urls_and_ids


driver = webdriver.Chrome(executable_path="./chromedriver") 
driver.implicitly_wait(.05) 

df_leagues_teams_names_urls_and_ids = pd.DataFrame(columns=['club_name', 'id_number', 'url_name'])


for url in league_urls:
    
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    leagues_teams_scraper(soup)
    
df_leagues_teams_names_urls_and_ids = df_leagues_teams_names_urls_and_ids[df_leagues_teams_names_urls_and_ids['club_name'] != '']
df_leagues_teams_names_urls_and_ids = df_leagues_teams_names_urls_and_ids.drop_duplicates(subset='id_number')

# Export to csv dataframe of df_leagues_teams_names_urls_and_ids
df_leagues_teams_names_urls_and_ids.to_csv('df_leagues_teams_names_urls_and_ids.csv')

# Check Length of df_leagues_teams_names_urls_and_ids
print(len(df_leagues_teams_names_urls_and_ids))

# Check head
df_leagues_teams_names_urls_and_ids.head()

driver.quit()

98


,club_name,id_number,url_name
1,FC Barcelona,131,fc-barcelona
4,Real Madrid,418,real-madrid
7,Atlético Madrid,13,atletico-madrid
10,Valencia CF,1049,fc-valencia
13,Sevilla FC,368,fc-sevilla


# Build out the function to insert team name and team id into url to scrap stats and market value page

In [49]:
# Create List of all Leagues' Team Ids using df_leagues_teams_names_urls_and_ids

ids = [df_leagues_teams_names_urls_and_ids.iloc[i,1] for i in range(0, len(df_leagues_teams_names_urls_and_ids))]
ids[:5]

['131', '418', '13', '1049', '368']

In [50]:
# Create List of all Leagues' Team's Names used in URL using df_leagues_teams_names_urls_and_ids

teams = [df_leagues_teams_names_urls_and_ids.iloc[i,2] for i in range(0, len(df_leagues_teams_names_urls_and_ids))]
teams[:5]

['fc-barcelona', 'real-madrid', 'atletico-madrid', 'fc-valencia', 'fc-sevilla']

# List of Years

In [51]:
# # Insert below Number of Years of Data you Want to Collect (Max is 20 years)
# # Will collected data from 2017 and previous

# num_of_years_wanted = 5

# list_of_twenty_years = ['1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010', '2011','2012', '2013', '2014','2015', '2016', '2017']
# list_of_years_being_used = [i for i in list_of_twenty_years][-num_of_years_wanted:]
# list_of_years_being_used


#### Currently Using This One

In [70]:
# For 2008 - 2012
num_of_years_wanted = 5
list_of_twenty_years = ['1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010', '2011','2012']
list_of_years_being_used = [i for i in list_of_twenty_years][-num_of_years_wanted:]
list_of_years_being_used

['2008', '2009', '2010', '2011', '2012']

In [120]:
# # For 2007
# num_of_years_wanted = 1
# list_of_twenty_years = ['2007']
# list_of_years_being_used = [i for i in list_of_twenty_years][-num_of_years_wanted:]
# list_of_years_being_used

# Base Urls 


# CHANGE HERE TO GET DATA FROM MORE YEARS

In [72]:
# Create a list of all the teams and their ids per each of the 10 years
####### change for loop iterable to whichever list of years that you want to do complete scrap on

for_loops_list_variable = list_of_years_being_used.copy()


list_of_all_teams_ids_years = []
for i in for_loops_list_variable:            ####### change this to whichever list of years that you want to do complete scrap on
    for (a, b) in zip(teams, ids):
        c = a, b + '',''+i
        list_of_all_teams_ids_years.append(c)
        
# Years Being Used In the function from here out
print(for_loops_list_variable)
        

['2008', '2009', '2010', '2011', '2012']


In [119]:
# # Uncomment to see list of info to build urls
# list_of_all_teams_ids_years

In [74]:
# This is how many individual pages of team data that will be collected

len(list_of_all_teams_ids_years) * 2

980

In [75]:
## Make list of URLS: url1 & url2

# url1 for players market values
list_url1 = []
for i in range(len(list_of_all_teams_ids_years)):
    url1 = "https://www.transfermarkt.co.uk/" + list_of_all_teams_ids_years[i][0] + "/kader/verein/" + list_of_all_teams_ids_years[i][1] + "/saison_id/" + list_of_all_teams_ids_years[i][2] + "/plus/1"
    list_url1.append(url1)
    # print(url1)  # in order to see the urls and click on them
    
    
# url2 for players stats
list_url2 = []
for i in range(len(list_of_all_teams_ids_years)):
    url2 = "https://www.transfermarkt.co.uk/" + list_of_all_teams_ids_years[i][0] + "/leistungsdaten/verein/" + list_of_all_teams_ids_years[i][1] + "/reldata/%26" + list_of_all_teams_ids_years[i][2] + "/plus/1"
    list_url2.append(url2)
     # print(url2) # in order to see the urls and click on them
    



In [76]:
print('Scrapper will run over ' + str(len(list_url1)) + ' Market Value Pages and ' + str(len(list_url2)) + ' Player Stats pages for each Club for a total of ' + str(len(list_url1) + len(list_url2)) + ' unique webpages.')



Scrapper will run over 490 Market Value Pages and 490 Player Stats pages for each Club for a total of 980 unique webpages.


#### Break

# Data Retrieval Function called get_data that takes two urls

In [36]:
# # Uncomment to check list_url1 and list_url2 for all
# for a, b in zip(list_url1, list_url2):
#      print(a, b)

In [37]:
# # Uncomment to see all list_url1
# list_url1

In [38]:
# # Uncomment to see all list_url2
# list_url2

In [41]:
# how many pages it'll go to, if fully run
len(list_url1)

490

# For 2013-2017 data scrap

In [44]:
# print('Scrapper will run over ' + str(len(list_url1)) + ' Market Value Pages and ' + str(len(list_url2)) + ' Player Stats pages for each Club for a total of ' + str(len(list_url1) + len(list_url2)) + ' unique webpages.')


Scrapper will run over 490 Market Value Pages and 490 Player Stats pages for each Club for a total of 980 unique webpages.


In [42]:
# # Run this to return the dataframe of info

# driver = webdriver.Chrome(executable_path="./chromedriver") 
# driver.implicitly_wait(5) 


# list_of_dfs = []

# for url1, url2 in zip(list_url1, list_url2):
    
#     try:
#         driver.get(url1)
#         soup1 = BeautifulSoup(driver.page_source, 'lxml')
#         sleep(.1)
#     except:
#         continue
    
#     try:
#         driver.get(url2)
#         soup2 = BeautifulSoup(driver.page_source, 'lxml')
#     except:
#         continue

#     list_of_dfs.append(get_data(soup1, soup2))

# df_master = pd.concat(list_of_dfs, ignore_index=True, sort=False)
# df_master.to_csv('df_master.csv')
# print(df_master.shape)
 

# driver.quit()
# df_master.head()

(14644, 28)


,player,current_team,current_year,birthdate_age,nationality,height_m,foot,date_joined,year_joined,signed_from,transfer_fee,contract_until,market_value,number,position,general_position,age,in_squad,appearances,goals,assists,yellow_card,second_yellow,red_cards,substituted_on,substituted_off,ppm,minutes_played
0,Víctor Valdés,fc-barcelona,2013,"Jan 14, 1982 (31)",Spain,"1,83 m",right,"Jul 1, 2002",2002,FC Barcelona B,-,-,£9.00m,1,Goalkeeper,Goalkeeper,31,40,34,-,-,2,-,-,-,1,"2,38",2.993'
1,Oier Olazábal,fc-barcelona,2013,"Sep 14, 1989 (23)",Spain,"1,89 m",left,"Jul 1, 2013",2013,Spain,-,-,£1.35m,25,Goalkeeper,Goalkeeper,23,21,Was not used during this season,-,-,-,-,-,-,-,0,-
2,José Manuel Pinto,fc-barcelona,2013,"Nov 8, 1975 (37)",Spain,"1,85 m",right,"Jul 1, 2008",2008,Celta de Vigo,Free,-,£270k,13,Goalkeeper,Goalkeeper,37,57,26,-,-,1,-,-,1,-,"2,04",2.317'
3,Gerard Piqué,fc-barcelona,2013,"Feb 2, 1987 (26)",Spain,"1,94 m",right,"Jul 1, 2008",2008,Manchester United,£4.50m,30.06.2022,£32.40m,3,Centre-Back,Defender,26,43,39,4,-,7,-,-,-,4,"2,21",3.393'
4,Marc Bartra,fc-barcelona,2013,"Jan 15, 1991 (22)",Spain,"1,84 m",right,"Jul 1, 2012",2012,FC Barcelona B,-,-,£5.40m,15,Centre-Back,Defender,22,51,30,2,1,4,-,-,6,1,"2,37",2.302'


# For 2008-2012 data scrap

In [77]:
print('Scrapper will run over ' + str(len(list_url1)) + ' Market Value Pages and ' + str(len(list_url2)) + ' Player Stats pages for each Club for a total of ' + str(len(list_url1) + len(list_url2)) + ' unique webpages.')



Scrapper will run over 490 Market Value Pages and 490 Player Stats pages for each Club for a total of 980 unique webpages.


In [79]:
# how many pages it'll go to, if fully run
len(list_url1)*2

980

In [80]:
# # Run this to return the dataframe of info for 2008 to 2012

# Run this to return the dataframe of info for 2008 to 2012

driver = webdriver.Chrome(executable_path="./chromedriver") 
driver.implicitly_wait(5) 


list_of_dfs = []

for url1, url2 in zip(list_url1, list_url2):
    
    try:
        driver.get(url1)
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        sleep(.1)
        driver.refresh()
    except:
        continue
    
    try:
        driver.get(url2)
        soup2 = BeautifulSoup(driver.page_source, 'lxml')
        sleep(.1)
        driver.refresh()
    except:
        continue

    list_of_dfs.append(get_data(soup1, soup2))

df_master = pd.concat(list_of_dfs, ignore_index=True, sort=False) # added sort=False here if reran
df_master.to_csv('df_master_2008_2012_uncleaned.csv')
print(df_master.shape)
 

driver.quit()
df_master.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(15860, 28)


,age,appearances,assists,birthdate_age,contract_until,current_team,current_year,date_joined,foot,general_position,goals,height_m,in_squad,market_value,minutes_played,nationality,number,player,position,ppm,red_cards,second_yellow,signed_from,substituted_off,substituted_on,transfer_fee,year_joined,yellow_card
0,26,49,-,"Jan 14, 1982 (26)",-,fc-barcelona,2008,"Jul 1, 2002",right,Goalkeeper,-,"1,83 m",52,£13.50m,4.410',Spain,1,Víctor Valdés,Goalkeeper,"2,35",-,-,FC Barcelona B,-,-,-,2002,3
1,32,11,-,"Nov 8, 1975 (32)",-,fc-barcelona,2008,"Jul 1, 2008",right,Goalkeeper,-,"1,85 m",51,£1.80m,990',Spain,13,José Manuel Pinto,Goalkeeper,"2,18",-,-,Spain,-,-,-,2008,1
2,29,1,-,"Mar 3, 1979 (29)",-,fc-barcelona,2008,"Jul 1, 2003",-,Goalkeeper,-,"1,83 m",20,£720k,90',Spain,25,Albert Jorquera,Goalkeeper,"0,00",-,-,FC Barcelona B,-,-,-,2003,-
3,18,1,-,"Sep 14, 1989 (18)",-,fc-barcelona,2008,"Aug 1, 2007",left,Goalkeeper,-,"1,89 m",2,£540k,90',Spain,-,Oier Olazábal,Goalkeeper,"0,00",-,-,FC Barcelona B,-,-,-,2007,-
4,30,45,3,"Apr 13, 1978 (30)",-,fc-barcelona,2008,"Jul 1, 1999",right,Defender,1,"1,78 m",49,£23.40m,3.768',Spain,5,Carles Puyol,Centre-Back,"2,29",-,-,FC Barcelona B,6,4,-,1999,8


In [82]:
df_master = pd.concat(list_of_dfs, ignore_index=True, sort=False)
df_master.to_csv('df_master_2008_2012_uncleaned.csv')
print(df_master.shape)
df_master.head()

(15860, 28)


,player,current_team,current_year,birthdate_age,nationality,height_m,foot,date_joined,year_joined,signed_from,transfer_fee,contract_until,market_value,number,position,general_position,age,in_squad,appearances,goals,assists,yellow_card,second_yellow,red_cards,substituted_on,substituted_off,ppm,minutes_played
0,Víctor Valdés,fc-barcelona,2008,"Jan 14, 1982 (26)",Spain,"1,83 m",right,"Jul 1, 2002",2002,FC Barcelona B,-,-,£13.50m,1,Goalkeeper,Goalkeeper,26,52,49,-,-,3,-,-,-,-,"2,35",4.410'
1,José Manuel Pinto,fc-barcelona,2008,"Nov 8, 1975 (32)",Spain,"1,85 m",right,"Jul 1, 2008",2008,Spain,-,-,£1.80m,13,Goalkeeper,Goalkeeper,32,51,11,-,-,1,-,-,-,-,"2,18",990'
2,Albert Jorquera,fc-barcelona,2008,"Mar 3, 1979 (29)",Spain,"1,83 m",-,"Jul 1, 2003",2003,FC Barcelona B,-,-,£720k,25,Goalkeeper,Goalkeeper,29,20,1,-,-,-,-,-,-,-,"0,00",90'
3,Oier Olazábal,fc-barcelona,2008,"Sep 14, 1989 (18)",Spain,"1,89 m",left,"Aug 1, 2007",2007,FC Barcelona B,-,-,£540k,-,Goalkeeper,Goalkeeper,18,2,1,-,-,-,-,-,-,-,"0,00",90'
4,Carles Puyol,fc-barcelona,2008,"Apr 13, 1978 (30)",Spain,"1,78 m",right,"Jul 1, 1999",1999,FC Barcelona B,-,-,£23.40m,5,Centre-Back,Defender,30,49,45,1,3,8,-,-,4,6,"2,29",3.768'


# END